In [138]:
#Deep learning, a subfield of machine learning that is a set of algorithms 
#that is inspired by the structure and function of the brain. 
#These algorithms are usually called Artificial Neural Networks (ANN).

#two key architecture decisions that you need to make to make your model: 
#how many layers you’re going to use and
#how many “hidden units” you will choose for each layer.

In [1]:
import tensorflow as tf

tf.__version__


'2.3.0'

In [31]:
#A multilayer perceptron (MLP) is a class of feedforward artificial neural

#Objective:  predict whether a wine is red or white by looking at 
#its chemical properties, such as volatile acidity or sulphates

#Source data: https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/

#I conbined data of red and white wine and creating new column as "wine" to
#identify type of wine

# Load libraries
import pandas as pd #package for manage dataframe
import numpy as np 
import matplotlib.pyplot as plt 
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score

In [3]:
# load dataset
winequality = pd.read_csv("winequality.csv")

In [4]:
#Display all results in the same cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [6]:
winequality.head()
winequality.shape
winequality.describe()
winequality.isnull().any() # False means no missing values
winequality.info()  #wine as categorical variable (object)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,wine
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,1
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1


(6497, 13)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,wine
count,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000
mean,7.215307,0.339666,0.318633,5.443235,0.056034,30.525319,115.744574,0.994697,3.218501,0.531268,10.491801,5.818378,0.246114
std,1.296434,0.164636,0.145318,4.757804,0.035034,17.749400,56.521855,0.002999,0.160787,0.148806,1.192712,0.873255,0.430779
min,3.800000,0.080000,0.000000,0.600000,0.009000,1.000000,6.000000,0.987110,2.720000,0.220000,8.000000,3.000000,0.000000
25%,6.400000,0.230000,0.250000,1.800000,0.038000,17.000000,77.000000,0.992340,3.110000,0.430000,9.500000,5.000000,0.000000
50%,7.000000,0.290000,0.310000,3.000000,0.047000,29.000000,118.000000,0.994890,3.210000,0.510000,10.300000,6.000000,0.000000
75%,7.700000,0.400000,0.390000,8.100000,0.065000,41.000000,156.000000,0.996990,3.320000,0.600000,11.300000,6.000000,0.000000
max,15.900000,1.580000,1.660000,65.800000,0.611000,289.000000,440.000000,1.038980,4.010000,2.000000,14.900000,9.000000,1.000000


fixed acidity           False
volatile acidity        False
citric acid             False
residual sugar          False
chlorides               False
free sulfur dioxide     False
total sulfur dioxide    False
density                 False
pH                      False
sulphates               False
alcohol                 False
quality                 False
wine                    False
dtype: bool

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         6497 non-null   float64
 1   volatile acidity      6497 non-null   float64
 2   citric acid           6497 non-null   float64
 3   residual sugar        6497 non-null   float64
 4   chlorides             6497 non-null   float64
 5   free sulfur dioxide   6497 non-null   float64
 6   total sulfur dioxide  6497 non-null   float64
 7   density               6497 non-null   float64
 8   pH                    6497 non-null   float64
 9   sulphates             6497 non-null   float64
 10  alcohol               6497 non-null   float64
 11  quality               6497 non-null   int64  
 12  wine                  6497 non-null   int64  
dtypes: float64(11), int64(2)
memory usage: 660.0 KB


In [7]:
winequality.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality', 'wine'],
      dtype='object')

In [10]:
# To divide the data into “attributes” (independent vatiables) and “label” (target or dependent variable)
X = winequality.drop('wine', axis='columns') #drop column wine
y = winequality['wine']

X.head()
y.head() #Red = 1 White = 0
X.shape

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


0    1
1    1
2    1
3    1
4    1
Name: wine, dtype: int64

(6497, 12)

In [11]:
# Split the data up in train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [12]:
X_train.shape
X_test.shape
y_train.shape
y_test.shape

(4352, 12)

(2145, 12)

(4352,)

(2145,)

In [13]:
#Standardize The Data

#Import StandardScaler from sklearn.preprocessing
from sklearn.preprocessing import StandardScaler

# Define the scaler 
scaler = StandardScaler().fit(X_train)

# Scale the train set
X_train = scaler.transform(X_train)

# Scale the test set
X_test = scaler.transform(X_test)

X_train
X.shape

array([[-7.51682345e-02, -1.33696832e+00,  8.58590678e-04, ...,
        -8.17593880e-01, -9.07381844e-01, -9.29111795e-01],
       [-3.09279988e-01, -1.33696832e+00, -1.37533507e-01, ...,
        -1.23015908e+00, -4.88519150e-01,  2.07659229e-01],
       [ 3.93055272e-01,  2.71544591e-01,  5.54426980e-01, ...,
        -1.43644168e+00,  2.02465702e+00,  2.07659229e-01],
       ...,
       [-6.21428992e-01,  2.09678710e-01, -8.29493994e-01, ...,
        -6.80072147e-01, -8.23609305e-01, -9.29111795e-01],
       [-5.43391741e-01, -4.70845982e-01,  1.24638747e+00, ...,
        -4.05028680e-01, -8.23609305e-01, -9.29111795e-01],
       [ 2.86901658e-03,  1.75632574e+00, -1.79823868e+00, ...,
         7.62973863e-02, -8.23609305e-01, -9.29111795e-01]])

(6497, 12)

In [14]:
#Import Sequential from keras.models
from tensorflow.keras.models import Sequential

In [15]:
#Import Dense from keras.layers
from tensorflow.keras.layers import Dense,Dropout,Activation, Flatten, Conv2D, MaxPooling2D

In [16]:
#Initialize the constructor
#To use the Keras Sequential model: it’s a linear stack of layers. 
model = Sequential()

# Add an input layer 
#In the first layer, the activation argument takes the value relu.
#12 hidden units; this means that the model will output arrays of shape (*, 12)
#This is is the dimensionality of the output space.

#If you would allow more hidden units, 
#your network will be able to learn more complex representations 
#but it will also be a more expensive operations 
#that can be prone to overfitting.

model.add(Dense(12, input_dim=12, activation='relu')) #X has 12 columns, see in X.shape

# Add one hidden layer 

#The intermediate layer also uses the relu activation function. 
#The output of this layer will be arrays of shape (*,8)

model.add(Dense(8, activation='relu'))

# Add an output layer 

#The network with a Dense layer of size 1. 
#The final layer will also use a sigmoid activation function 
#so that your output is actually a probability; 
#This means that this will result in a score between 0 and 1, 
#indicating how likely the sample is to have the target “1”, or how likely the wine is to be red.

model.add(Dense(1, activation='sigmoid'))


In [17]:
#In compiling, you configure the model with the adam optimizer 
#and the binary_crossentropy loss function. 

#Some of the most popular optimization algorithms used are
#the Stochastic Gradient Descent (SGD), ADAM and RMSprop. 

#The choice for a loss function:
#MSE for regression
#binary_crossentropy for the binary classification problem of determining whether a wine is red or white. 
#multi-class classification, you’ll make use of categorical_crossentropy.

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [18]:
model.fit(X_train, y_train, epochs=5)
#2 epochs or iterations
#verbose = 1, you indicate that you want to see progress bar logging.

Epoch 1/5
136/136 [==============================] - 0s 793us/step - loss: 0.4854 - accuracy: 0.8817
Epoch 2/5
136/136 [==============================] - 0s 801us/step - loss: 0.1385 - accuracy: 0.9853
Epoch 3/5
136/136 [==============================] - 0s 899us/step - loss: 0.0502 - accuracy: 0.9931
Epoch 4/5
136/136 [==============================] - 0s 963us/step - loss: 0.0342 - accuracy: 0.9947
Epoch 5/5
136/136 [==============================] - 0s 1ms/step - loss: 0.0285 - accuracy: 0.9952


In [19]:
y_pred = model.predict_classes(X_test)
y_pred1 = model.predict(X_test)
y_pred[:10]
y_pred1[:10]

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


array([[0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0]])

array([[1.3082922e-03],
       [8.3226514e-01],
       [2.4483502e-03],
       [3.6489964e-04],
       [2.0071864e-04],
       [9.7756916e-01],
       [1.7847121e-03],
       [2.0865202e-03],
       [9.3460679e-01],
       [2.5348663e-03]], dtype=float32)

In [20]:
#Accuracy

score = model.evaluate(X_test, y_test)

print(score)

#Accuracy = 0.9888

68/68 [==============================] - 0s 626us/step - loss: 0.0404 - accuracy: 0.9888
[0.040438540279865265, 0.988811194896698]


In [41]:
# Import the modules from `sklearn.metrics`
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, cohen_kappa_score

# Confusion matrix
confusion_matrix(y_test, y_pred)

array([[1577,   11],
       [  13,  544]], dtype=int64)

In [34]:
X_test.shape

(2145, 12)

In [28]:
auc = roc_auc_score(y_test, y_pred)
print("AUC:", np.round(auc, 6))

AUC: 0.984867


In [42]:
# Precision 

#Precision is a measure of a classifier’s exactness. 
#The higher the precision, the more accurate the classifier

precision_score(y_test, y_pred) 
accuracy_score(y_test, y_pred) #(1577+544)/2145

0.9801801801801802

0.9888111888111888

In [39]:
# Recall

#Recall is a measure of a classifier’s completeness. 
#The higher the recall, the more cases the classifier covers.

recall_score(y_test, y_pred)

0.9766606822262118

In [43]:
# F1 score

#The F1 Score or F-score is a weighted average of precision and recall.

f1_score(y_test,y_pred)

0.9784172661870504

In [44]:
# Cohen's kappa

#The Kappa or Cohen’s kappa is the classification accuracy normalized 
#by the imbalance of the classes in the data.

cohen_kappa_score(y_test, y_pred)

0.970865379717822